# CAR-Conv2

In [1]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. 

In [2]:
TRAIN_DIR = 'train_car/'
TEST_DIR = 'test/'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'twocars-{}-{}.model'.format(LR, '2conv-basic')

In [3]:
def label_img(img):
    word_label = img.split('.')[-3]
    
    if word_label == 'audia4': return [1,0,0,0]
   
    elif word_label == 'ferrari': return [0,1,0,0]
    
    elif word_label == 'benzslk': return [0,0,1,0]
    
    elif word_label == 'bmw325': return [0,0,0,1]

In [4]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data_car.npy', training_data)
    return training_data

In [5]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('test_data_car.npy', testing_data)
    return testing_data

In [6]:
#train_data = create_train_data()
train_data = np.load('train_data_car.npy')

In [7]:
b = np.load('train_data_car.npy')
print(b)

[[ array([[255, 255, 188, ...,  90, 255, 255],
       [255, 255, 195, ...,  83, 254, 255],
       [255, 255, 159, ...,  81, 253, 255],
       ..., 
       [255, 254, 130, ..., 130, 250, 255],
       [255, 255, 137, ..., 191, 253, 255],
       [255, 248, 128, ..., 190, 253, 255]], dtype=uint8)
  array([1, 0, 0, 0])]
 [ array([[100, 148,  96, ...,  82,  59,  90],
       [ 62, 175, 244, ...,  79,  79,  79],
       [112,  85,  84, ...,  56,  61,  62],
       ..., 
       [ 49,  49,  56, ...,  96,  96,  98],
       [ 60,  66,  44, ...,  96,  94,  96],
       [ 41,  52,  42, ...,  96,  94,  92]], dtype=uint8)
  array([0, 0, 1, 0])]
 [ array([[ 59,  43,  47, ..., 103, 153,  68],
       [ 53,  43,  52, ..., 107, 154,  91],
       [ 49,  49,  60, ..., 103, 157,  94],
       ..., 
       [115, 151,  97, ...,  49,  67,  86],
       [139, 110,  71, ..., 134, 130, 125],
       [146,  93, 141, ...,  82, 111,  76]], dtype=uint8)
  array([0, 1, 0, 0])]
 ..., 
 [ array([[209, 226, 236, ..., 142,  40, 1

In [8]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
#How many class should be push
convnet = fully_connected(convnet, 4, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


/Library/anaconda3/envs/tflearn/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Scipy not supported!
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [9]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

In [10]:
train = train_data[:500]
test = train_data[-500:]

In [11]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

In [12]:
model.fit({'input': X}, {'targets': Y}, n_epoch=50, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 399  | total loss: 14.30695 | time: 1.270s
| Adam | epoch: 050 | loss: 14.30695 - acc: 0.3787 -- iter: 448/500
Training Step: 400  | total loss: 14.38733 | time: 2.465s
| Adam | epoch: 050 | loss: 14.38733 - acc: 0.3752 | val_loss: 14.82865 - val_acc: 0.3560 -- iter: 500/500
--


# tensorboard --logdir=